In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_selection import mutual_info_classif, RFE, SelectKBest, SelectFromModel, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from category_encoders import HashingEncoder, CountEncoder
from scipy.stats import skew, kurtosis
from sklearn.base import BaseEstimator, TransformerMixin
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier

# Suppress warnings and RuntimeWarnings
warnings.filterwarnings('ignore')


class DatetimeConvertCyclical(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.time_periods = {'second': 24 * 60 * 60,
                             'minute': 24 * 60,
                             'hour': 24,
                             'day': 30,
                             'dayofweek': 7,
                             'month': 12}

    def fit(self, X, y=None):
        return self

    def transform(self, X, time_col='timestamp'):
        for period, value in self.time_periods.items():
            X[period] = getattr(X[time_col].dt, period)

            X[f'{time_col}_sin_' +
                period] = np.sin(2 * np.pi * X[period] / value)
            X[f'{time_col}_cos_' +
                period] = np.cos(2 * np.pi * X[period] / value)

            X.drop(str(period), axis=1, inplace=True)

        return X


class DataProcessor:
    def __init__(self):
        self.encoders = {}
        self.scaler = StandardScaler()

    @staticmethod
    def _convert_time_columns(data):
        data['start_time'] = pd.to_datetime(
            data['start_time'], format='%Y%m%d%H%M%S')
        data['end_time'] = pd.to_datetime(
            data['end_time'], format='%Y%m%d%H%M%S')
        data['update_time'] = pd.to_datetime(
            data['update_time'], format='%Y-%m-%d %H:%M:%S')
        data['date'] = pd.to_datetime(
            data['date'], format='%Y-%m-%d')
        data['date_c'] = pd.to_datetime(
            data['date_c'], format='%Y%m%d')
        return data

    def _extract_time_features(self, data):
        cyclical_transformer = DatetimeConvertCyclical()
        data = cyclical_transformer.transform(data, 'date')
        data['start_hour'] = data['start_time'].dt.hour
        data['start_dayofweek'] = data['start_time'].dt.dayofweek
        data['is_weekend'] = data['start_dayofweek'].apply(
            lambda x: 1 if x >= 5 else 0)
        data['is_working_hour'] = data['start_hour'].apply(
            lambda x: 1 if 9 <= x <= 18 else 0)
        return data

    def _encode_categorical_features(self, data, encoding_config, fit=True):
        for feature, encoding_method in encoding_config.items():
            if encoding_method == 'onehot':
                if fit:
                    encoder = OneHotEncoder(
                        sparse_output=False, handle_unknown='ignore')
                    encoded = encoder.fit_transform(data[[feature]])
                    self.encoders[feature] = encoder
                else:
                    encoder = self.encoders.get(feature)
                    if encoder:
                        encoded = encoder.transform(data[[feature]])
                    else:
                        continue  # Skip if encoder is not available
                encoded_df = pd.DataFrame(
                    encoded, columns=[f"{feature}_{cat}" for cat in encoder.categories_[0]])
                data = pd.concat([data, encoded_df], axis=1)
                data.drop(columns=[feature], inplace=True)
            elif encoding_method == 'label':
                if fit:
                    encoder = OrdinalEncoder(
                        handle_unknown='use_encoded_value', unknown_value=-1)
                    encoded = encoder.fit_transform(data[[feature]])
                    self.encoders[feature] = encoder
                else:
                    encoder = self.encoders.get(feature)
                    if encoder:
                        encoded = encoder.transform(data[[feature]])
                    else:
                        continue  # Skip if encoder is not available
                data[feature] = encoded
            elif encoding_method == 'hash':
                if fit:
                    encoder = HashingEncoder()
                    encoded = encoder.fit_transform(data[[feature]])
                    self.encoders[feature] = encoder
                else:
                    encoder = self.encoders.get(feature)
                    if encoder:
                        encoded = encoder.transform(data[[feature]])
                    else:
                        continue  # Skip if encoder is not available
                data = pd.concat([data, encoded], axis=1)
            elif encoding_method == 'count':
                if fit:
                    encoder = CountEncoder()
                    encoded = encoder.fit_transform(data[[feature]])
                    self.encoders[feature] = encoder
                else:
                    encoder = self.encoders.get(feature)
                    if encoder:
                        encoded = encoder.transform(data[[feature]])
                    else:
                        encoded = data[feature].map(
                            data[feature].value_counts())  # Frequency encoding
                data[feature] = encoded
            elif encoding_method == 'frequency':
                if fit:
                    freq = data[feature].value_counts() / len(data)
                    self.encoders[feature] = freq
                else:
                    freq = self.encoders.get(
                        feature, data[feature].value_counts() / len(data))
                data[feature] = data[feature].map(freq).fillna(0)
            elif encoding_method == 'labelcount':
                if fit:
                    encoder = CountEncoder(normalize=True)
                    encoded = encoder.fit_transform(data[[feature]])
                    self.encoders[feature] = encoder
                else:
                    encoder = self.encoders.get(feature)
                    if encoder:
                        encoded = encoder.transform(data[[feature]])
                    else:
                        continue  # Skip if encoder is not available
                data[feature] = encoded
            elif encoding_method == 'tfidf':
                vectorizer = self.tfidf_vectorizers.get(
                    feature, TfidfVectorizer())
                if fit:
                    encoded = vectorizer.fit_transform(
                        data[feature].astype(str)).toarray()
                    self.tfidf_vectorizers[feature] = vectorizer
                else:
                    encoded = vectorizer.transform(
                        data[feature].astype(str)).toarray()
                encoded_df = pd.DataFrame(
                    encoded, columns=[f"tfidf_{feature}_{i}" for i in range(encoded.shape[1])])
                data = pd.concat([data, encoded_df], axis=1).drop(
                    columns=[feature])
                continue
            elif encoding_method == 'countvector':
                vectorizer = self.count_vectorizers.get(
                    feature, CountVectorizer())
                if fit:
                    encoded = vectorizer.fit_transform(
                        data[feature].astype(str)).toarray()
                    self.count_vectorizers[feature] = vectorizer
                else:
                    encoded = vectorizer.transform(
                        data[feature].astype(str)).toarray()
                encoded_df = pd.DataFrame(
                    encoded, columns=[f"{feature}_count_{i}" for i in range(encoded.shape[1])])
                data = pd.concat([data, encoded_df], axis=1).drop(
                    columns=[feature])
                continue
            else:
                raise ValueError(f"Unknown encoding method: {encoding_method}")
        return data

    @staticmethod
    def _statistical_features(df, feature):
        agg_funcs = ['sum', 'mean', 'max', 'min', 'std',
                     'var', 'median', 'nunique', 'size', 'count']
        agg_funcs += [
            ('skew', lambda x: skew(x) if len(x) > 1 else np.nan),
            ('kurt', lambda x: kurtosis(x) if len(x) > 1 else np.nan),
            ('quantile_25', lambda x: x.quantile(0.25)),
            ('quantile_75', lambda x: x.quantile(0.75)),
            ('mode', lambda x: x.mode().iloc[0]
             if not x.mode().empty else np.nan)
        ]
        return df.groupby('msisdn')[feature].agg(agg_funcs).add_prefix(f'{feature}_')

    @staticmethod
    def _aggregate_numerical_features(data, numerical_features):
        user_aggregated_data = pd.DataFrame()
        for feature in numerical_features:
            feature_stats = DataProcessor._statistical_features(data, feature)
            if user_aggregated_data.empty:
                user_aggregated_data = feature_stats
            else:
                user_aggregated_data = user_aggregated_data.join(
                    feature_stats, how='outer')
        return user_aggregated_data

    @staticmethod
    def _aggregate_location_features(data, location_features):
        user_aggregated_data = pd.DataFrame()
        for feature in location_features:
            feature_stats = data.groupby(
                'msisdn')[feature].agg(['nunique', 'count'])
            feature_stats.columns = [
                f'{feature}_{stat}' for stat in feature_stats.columns]
            if user_aggregated_data.empty:
                user_aggregated_data = feature_stats
            else:
                user_aggregated_data = user_aggregated_data.join(
                    feature_stats, how='outer')
        return user_aggregated_data

    @staticmethod
    def _aggregate_categorical_frequencies(data, categorical_features, user_aggregated_data):

        for feature in categorical_features:
            frequency = data.groupby(
                ['msisdn', feature]).size().unstack(fill_value=0)
            normalized_frequency = frequency.div(frequency.sum(axis=1), axis=0)
            normalized_frequency.columns = [
                f"{feature}_{col}_freq" for col in normalized_frequency.columns]
            user_aggregated_data = user_aggregated_data.join(
                normalized_frequency, how='left')
        return user_aggregated_data

    @staticmethod
    def _aggregate_differential_features(data, numeric_features):
        for feature in numeric_features:
            data[f'{feature}_diff'] = data.groupby(
                'msisdn')[feature].diff().fillna(0)
            data[f'{feature}_diff2'] = data.groupby(
                'msisdn')[feature].diff(2).fillna(0)

        diff_agg_funcs = {f'{feature}_diff': [
            'mean', 'std'] for feature in numeric_features}
        diff_agg_funcs.update(
            {f'{feature}_diff2': ['mean', 'std'] for feature in numeric_features})

        diff_aggregated_data = data.groupby('msisdn').agg(diff_agg_funcs)
        diff_aggregated_data.columns = [
            '_'.join(map(str, col)).strip() for col in diff_aggregated_data.columns.values]

        return diff_aggregated_data

    def _extract_advanced_features(self, data):

        call_stats = data.groupby('msisdn').agg({
            'msisdn': ['count', 'nunique']
        })
        call_stats.columns = ['_'.join(col).strip()
                              for col in call_stats.columns.values]
        call_stats['avg_calls_per_person'] = call_stats['msisdn_count'] / \
            call_stats['msisdn_nunique']

        # 每个号码的通话最多的地市和区县
        most_common_city = data.groupby('msisdn')['phone1_loc_city'].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        most_common_county = data.groupby('msisdn')['phone1_loc_province'].agg(
            lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
        # 每天打电话最早小时数、最晚小时数、每天打电话的个数
        data['date'] = data['start_time'].dt.date
        daily_stats = data.groupby(['msisdn', 'date']).agg({
            'start_hour': ['min', 'max'],
            'call_duration': 'count'
        })
        daily_stats.columns = ['_'.join(col).strip()
                               for col in daily_stats.columns.values]
        daily_stats.reset_index(inplace=True)

        daily_stats_agg = daily_stats.groupby('msisdn').agg({
            'start_hour_min': ['mean', 'std'],
            'start_hour_max': ['mean', 'std'],
            'call_duration_count': ['mean', 'std']
        })
        daily_stats_agg.columns = [
            '_'.join(col).strip() for col in daily_stats_agg.columns.values]

        # 合并所有新特征
        advanced_features = pd.concat(
            [call_stats, most_common_city, most_common_county, daily_stats_agg], axis=1)
        advanced_features.columns = [
            f"advanced_{col}" for col in advanced_features.columns]

        return advanced_features

    @staticmethod
    def _binary_operations(user_aggregated_data):
        if 'cfee_sum' in user_aggregated_data.columns and 'lfee_sum' in user_aggregated_data.columns:
            user_aggregated_data['cfee_lfee_sum'] = user_aggregated_data['cfee_sum'] + \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_diff'] = user_aggregated_data['cfee_sum'] - \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_prod'] = user_aggregated_data['cfee_sum'] * \
                user_aggregated_data['lfee_sum']
            user_aggregated_data['cfee_lfee_ratio'] = user_aggregated_data['cfee_sum'] / (
                user_aggregated_data['lfee_sum'] + 1e-6)
        return user_aggregated_data

    def preprocess_and_aggregate(self, data, label_data=None, is_validation=False, fit_columns=None, encoding_config=None):
        data = self._convert_time_columns(data)
        data = self._extract_time_features(data)
        numerical_features = ['call_duration', 'cfee',
                              'lfee', 'start_hour', 'start_dayofweek']
        numerical_features += [col for col in data.columns if col.startswith(
            'date_sin_') or col.startswith('date_cos_')]
        differential_features = ['call_duration', 'cfee', 'lfee']
        categorical_features = [
            'call_event', 'roam_type', 'long_type1', 'ismultimedia',
            'is_weekend', 'is_working_hour', 'is_suspect'
        ]
        location_features = [
            'home_area_code', 'visit_area_code', 'called_home_code', 'called_code',
            'phone1_loc_city', 'phone1_loc_province', 'phone2_loc_city', 'phone2_loc_province'
        ]
        suspect_types = {3, 5, 6, 9, 11, 12, 17}
        data['is_suspect'] = data['phone1_type'].apply(
            lambda x: 1 if x in suspect_types else 0)

        data = self._encode_categorical_features(
            data, encoding_config, fit=not is_validation)

        user_aggregated_data = self._aggregate_numerical_features(
            data, numerical_features)
        user_aggregated_data = user_aggregated_data.join(
            self._aggregate_location_features(data, location_features), how='outer')
        user_aggregated_data = self._aggregate_categorical_frequencies(
            data, categorical_features, user_aggregated_data)
        user_aggregated_data = user_aggregated_data.join(
            self._aggregate_differential_features(data, differential_features), how='left')

        user_aggregated_data = self._binary_operations(user_aggregated_data)
        advanced_features = self._extract_advanced_features(data)
        user_aggregated_data = user_aggregated_data.join(
            advanced_features, how='left')

        user_aggregated_data.fillna(-1, inplace=True)
        user_aggregated_data.reset_index(inplace=True)

        if not is_validation and label_data is not None:
            user_aggregated_data = user_aggregated_data.merge(
                label_data, on='msisdn', how='left')

        if not is_validation:
            numerical_features = [
                col for col in user_aggregated_data.columns if col not in ['msisdn', 'is_sa']]
            user_aggregated_data[numerical_features] = self.scaler.fit_transform(
                user_aggregated_data[numerical_features])
            return user_aggregated_data, numerical_features, self.scaler
        else:
            user_aggregated_data[fit_columns] = self.scaler.transform(
                user_aggregated_data[fit_columns])
            return user_aggregated_data


def feature_selection(train_data, label_column='is_sa', k=20):
    X = train_data.drop(columns=['msisdn', label_column])
    y = train_data[label_column]

    def select_features(model, X, y, k):
        model.fit(X, y)
        feature_scores = pd.Series(model.feature_importances_, index=X.columns)
        return feature_scores.nlargest(k).index.tolist()

    xgb_selected_features = select_features(XGBClassifier(n_jobs=-1), X, y, k)
    rf_selected_features = select_features(
        RandomForestClassifier(n_jobs=-1), X, y, k)
    gbdt_selected_features = select_features(
        GradientBoostingClassifier(), X, y, k)
    et_selected_features = select_features(
        ExtraTreesClassifier(n_jobs=-1), X, y, k)

    select_k_best = SelectKBest(score_func=mutual_info_classif, k=k)
    select_k_best.fit(X, y)
    mutual_info_selected_features = X.columns[select_k_best.get_support(
    )].tolist()

    select_k_best = SelectKBest(score_func=f_classif, k=k)
    select_k_best.fit(X, y)
    f_classif_selected_features = X.columns[select_k_best.get_support(
    )].tolist()

    rfe_lr = LogisticRegression(random_state=42)
    rfe = RFE(rfe_lr, n_features_to_select=k)
    rfe.fit(X, y)
    rfe_lr_selected_features = X.columns[rfe.support_].tolist()

    rfe_svc = LinearSVC()
    rfe = RFE(rfe_svc, n_features_to_select=k)
    rfe.fit(X, y)
    rfe_svc_selected_features = X.columns[rfe.support_].tolist()

    corr_coef_selected_features = X.corrwith(
        y).abs().nlargest(k).index.tolist()

    all_selected_features = (
        xgb_selected_features + rf_selected_features + gbdt_selected_features + et_selected_features +
        mutual_info_selected_features + f_classif_selected_features +
        rfe_lr_selected_features + rfe_svc_selected_features +
        corr_coef_selected_features
    )

    combined_features = pd.Series(
        all_selected_features).value_counts().nlargest(k).index.tolist()
    return combined_features


# Encoding configuration
encoding_config = {
    'call_event': 'label',
    'other_party': 'label',
    'ismultimedia': 'label',
    'home_area_code': 'frequency',
    'visit_area_code': 'frequency',
    'called_home_code': 'frequency',
    'called_code': 'frequency',
    'a_serv_type': 'label',
    'long_type1': 'label',
    'roam_type': 'label',
    'a_product_id': 'tfidf',  # or 'countvector'
    'phone1_type': 'tfidf',  # or 'countvector'
    'phone2_type': 'tfidf',  # or 'countvector'
    'phone1_loc_city': 'tfidf',  # or 'countvector'
    'phone1_loc_province': 'tfidf',  # or 'countvector'
    'phone2_loc_city': 'tfidf',  # or 'countvector'
    'phone2_loc_province': 'tfidf',  # or 'countvector'
    'a_product_id': 'count',
    'phone1_type': 'label',
    'phone2_type': 'label',
    'phone1_loc_city': 'frequency',
    'phone1_loc_province': 'frequency',
    'phone2_loc_city': 'frequency',
    'phone2_loc_province': 'frequency',
    'is_weekend': 'label',
    'is_working_hour': 'label'
}


In [2]:
# 读取数据
train_set_res = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/train.csv', low_memory=False)
train_set_ans = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/labels.csv', low_memory=False)
validation_set_res = pd.read_csv(
    '/home/hwxu/Projects/Competition/Telecom/Input/raw/val.csv', low_memory=False)

In [3]:
def main(n_features_to_keep=40):
    # 实例化数据处理器
    data_processor = DataProcessor()
    # 处理训练集
    train_data, fit_columns, scaler = data_processor.preprocess_and_aggregate(
        train_set_res, train_set_ans, is_validation=False, encoding_config=encoding_config)

    print(f"num features: {train_data.shape[1]}")
    print(f"Constructed features: {train_data.columns.tolist()}")

    # 选择特征
    selected_features = feature_selection(train_data, k=n_features_to_keep)
    train_data = train_data[['msisdn'] + selected_features + ['is_sa']]

    print(f"num selected features: {len(selected_features)}")
    print(f"Selected features: {selected_features}")
    # 处理验证集
    validation_data = data_processor.preprocess_and_aggregate(
        validation_set_res, is_validation=True, fit_columns=fit_columns, encoding_config=encoding_config)

    # 选择特征（根据训练集选择的特征）
    validation_data = validation_data[['msisdn'] + selected_features]

    # 输出处理后的训练集和验证集
    train_data.to_csv(
        f'/home/hwxu/Projects/Competition/Telecom/Input/processed/train{n_features_to_keep}.csv', index=False)
    validation_data.to_csv(
        f'/home/hwxu/Projects/Competition/Telecom/Input/processed/val{n_features_to_keep}.csv', index=False)
    print(f'Shape of train data: {train_data.shape}, shape of validation data: {validation_data.shape}')


In [4]:
main(n_features_to_keep=175)

num features: 319
Constructed features: ['msisdn', 'call_duration_sum', 'call_duration_mean', 'call_duration_max', 'call_duration_min', 'call_duration_std', 'call_duration_var', 'call_duration_median', 'call_duration_nunique', 'call_duration_size', 'call_duration_count', 'call_duration_skew', 'call_duration_kurt', 'call_duration_quantile_25', 'call_duration_quantile_75', 'call_duration_mode', 'cfee_sum', 'cfee_mean', 'cfee_max', 'cfee_min', 'cfee_std', 'cfee_var', 'cfee_median', 'cfee_nunique', 'cfee_size', 'cfee_count', 'cfee_skew', 'cfee_kurt', 'cfee_quantile_25', 'cfee_quantile_75', 'cfee_mode', 'lfee_sum', 'lfee_mean', 'lfee_max', 'lfee_min', 'lfee_std', 'lfee_var', 'lfee_median', 'lfee_nunique', 'lfee_size', 'lfee_count', 'lfee_skew', 'lfee_kurt', 'lfee_quantile_25', 'lfee_quantile_75', 'lfee_mode', 'start_hour_sum', 'start_hour_mean', 'start_hour_max', 'start_hour_min', 'start_hour_std', 'start_hour_var', 'start_hour_median', 'start_hour_nunique', 'start_hour_size', 'start_hour_c

In [5]:
main(n_features_to_keep=180)

num features: 318
Constructed features: ['msisdn', 'call_duration_sum', 'call_duration_mean', 'call_duration_max', 'call_duration_min', 'call_duration_std', 'call_duration_var', 'call_duration_median', 'call_duration_nunique', 'call_duration_size', 'call_duration_count', 'call_duration_skew', 'call_duration_kurt', 'call_duration_quantile_25', 'call_duration_quantile_75', 'call_duration_mode', 'cfee_sum', 'cfee_mean', 'cfee_max', 'cfee_min', 'cfee_std', 'cfee_var', 'cfee_median', 'cfee_nunique', 'cfee_size', 'cfee_count', 'cfee_skew', 'cfee_kurt', 'cfee_quantile_25', 'cfee_quantile_75', 'cfee_mode', 'lfee_sum', 'lfee_mean', 'lfee_max', 'lfee_min', 'lfee_std', 'lfee_var', 'lfee_median', 'lfee_nunique', 'lfee_size', 'lfee_count', 'lfee_skew', 'lfee_kurt', 'lfee_quantile_25', 'lfee_quantile_75', 'lfee_mode', 'start_hour_sum', 'start_hour_mean', 'start_hour_max', 'start_hour_min', 'start_hour_std', 'start_hour_var', 'start_hour_median', 'start_hour_nunique', 'start_hour_size', 'start_hour_c